In [ ]:
import pyodbc
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import re

from matplotlib.ticker import MaxNLocator

In [ ]:
import os
from os.path import join, dirname
from dotenv import load_dotenv
dotenv_path = join(dirname(__file__), '.env')
load_dotenv(dotenv_path)

username = os.environ.get("USERNAME")
password = os.environ.get("PASSWORD")

In [ ]:
# Define the connection parameters
server = '80.70.181.166'  # Replace with your server name
database = 'NV_SAP_DWH'  # Your database name
username = username   # Your username
password = password   # Your password

# Create a connection string
connection_string = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'

In [ ]:
conn = pyodbc.connect(connection_string)

In [ ]:
# Create a cursor object
cursor = conn.cursor()

table_names = ["DIM_ITEM", "FACT_130_OUTGOING_INVOICE"]

dfs = {}

for table in table_names:
    query = f'SELECT * FROM {table}'
    cursor.execute(query)
    columns = [column[0] for column in cursor.description]
    rows = cursor.fetchall()
    transformed_rows = []
    for row in rows:
        transformed_rows.append(list(row))
    dfs[table] = pd.DataFrame(transformed_rows, columns=columns)

conn.close()

In [ ]:
invoice = dfs["FACT_130_OUTGOING_INVOICE"]

In [ ]:
# Convert the date column to datetime format
invoice['date'] = pd.to_datetime(invoice['OINV_INVOICE_DATE'], format='%d.%m.%Y')

# Extract month and year and create a new column for aggregation
invoice['month_year'] = invoice['date'].dt.to_period('M')

In [ ]:

monthly_data_inv = invoice.groupby('month_year')[['OINV_INVOICE_NET_VALUE', 'OINV_INVOICE_QUANTITY']].sum().reset_index()
monthly_data_inv['month_year'] = monthly_data_inv['month_year'].astype(str)

In [ ]:
# Create the figure and the subplots
fig, axs = plt.subplots(2, 1, figsize=(9, 5))

# Plot the first bar chart for net value
sns.barplot(ax=axs[0], x='month_year', y='OINV_INVOICE_NET_VALUE', data=monthly_data_inv)
axs[0].set_xlabel('Month-Year')
axs[0].set_ylabel('Total Net Value')
axs[0].set_title('Monthly Aggregated Net Value')
axs[0].tick_params(axis='x', rotation=45)

# Use MaxNLocator to reduce the number of x-ticks to 10
axs[0].xaxis.set_major_locator(MaxNLocator(nbins=10))

# Plot the second bar chart for quantity
sns.barplot(ax=axs[1], x='month_year', y='OINV_INVOICE_QUANTITY', data=monthly_data_inv)
axs[1].set_xlabel('Month-Year')
axs[1].set_ylabel('Total Quantity')
axs[1].set_title('Monthly Aggregated Quantity')
axs[1].tick_params(axis='x', rotation=45)

# Use MaxNLocator to reduce the number of x-ticks to 10
axs[1].xaxis.set_major_locator(MaxNLocator(nbins=10))

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
items = dfs["DIM_ITEM"]; items

In [ ]:
set(list(items["SL_DISTCHA_TEXT"]))

In [ ]:
set(list(items["SL_SAORG_TEXT"]))

In [ ]:
set(list(items["SL_BR_NO"]))

In [ ]:
invoice["item_id"] = invoice["OINV_ITEM_ID"]
items["item_id"] = items["SL_ITEM_ID"]

In [ ]:
all_item_ids_match = invoice['item_id'].isin(items['item_id'])
missing_item_ids_orders = invoice[~all_item_ids_match]

In [ ]:
print("Orders with item_ids not found in items dataframe:")
print(missing_item_ids_orders["item_id"])

In [ ]:
merged = pd.merge(invoice, items, on="item_id")

In [ ]:
print(merged.isnull().sum())

In [ ]:
final_customer_merged = merged[merged["SL_DISTCHA_TEXT"] == "Final customer sales"]

In [ ]:
final_customer_merged["OINV_INVOICE_NET_VALUE"]

In [ ]:
for name in set(list(items["SL_SAORG_TEXT"])):
    print(name)
    print(len(merged[(merged["SL_SAORG_TEXT"] == name) & (merged["SL_DISTCHA_TEXT"] == "Final customer sales")]))

In [ ]:
len(invoice)

In [ ]:
merged["OINV_SECTOR_NO"]

In [ ]:
len(items)

In [ ]:
invoice["OINV_SECTOR_NO"].sample(10)